In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bank-marketing-response-predict/submission.csv
/kaggle/input/bank-marketing-response-predict/test_label.csv
/kaggle/input/bank-marketing-response-predict/train.csv
/kaggle/input/bank-marketing-response-predict/test.csv


In [2]:
train = pd.read_csv('../input/bank-marketing-response-predict/train.csv')
test = pd.read_csv('../input/bank-marketing-response-predict/test.csv')

display(train.head(3))
display(test.head(3))

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome
0,53608,32,management,single,tertiary,no,12569,no,no,cellular,1,jul,2,295,2,success
1,51055,25,services,single,secondary,no,801,no,no,cellular,5,jun,2,-1,0,unknown
2,52573,46,blue-collar,married,secondary,no,1728,yes,no,unknown,26,may,2,-1,0,unknown


In [3]:
y_train = train['y']
test_id = test['ID']
x_train = train.drop(columns = ['ID','y'])
x_test = test.drop(columns = ['ID'])

In [4]:
y_train = y_train.replace('no', 0).replace('yes', 1)

In [5]:
print(x_train.info())
print(x_test.info())
print(x_train.shape)
print(x_test.shape)
print(x_train.isna().sum())
print(x_test.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12870 entries, 0 to 12869
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        12870 non-null  int64 
 1   job        12870 non-null  object
 2   marital    12870 non-null  object
 3   education  12870 non-null  object
 4   default    12870 non-null  object
 5   balance    12870 non-null  int64 
 6   housing    12870 non-null  object
 7   loan       12870 non-null  object
 8   contact    12870 non-null  object
 9   day        12870 non-null  int64 
 10  month      12870 non-null  object
 11  campaign   12870 non-null  int64 
 12  pdays      12870 non-null  int64 
 13  previous   12870 non-null  int64 
 14  poutcome   12870 non-null  object
dtypes: int64(6), object(9)
memory usage: 1.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4291 entries, 0 to 4290
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     -----------

In [6]:
print(x_train.select_dtypes(exclude = 'object').columns)
print(x_test.select_dtypes(exclude = 'object').columns)

Index(['age', 'balance', 'day', 'campaign', 'pdays', 'previous'], dtype='object')
Index(['age', 'balance', 'day', 'campaign', 'pdays', 'previous'], dtype='object')


In [7]:
x_train

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome
0,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown
1,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown
2,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown
3,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown
4,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12865,47,technician,married,secondary,no,1167,yes,no,cellular,30,apr,1,87,5,failure
12866,31,unknown,married,secondary,no,111,no,no,cellular,21,nov,2,93,2,failure
12867,37,unemployed,single,tertiary,no,1316,yes,no,cellular,18,nov,1,172,2,failure
12868,42,management,married,tertiary,no,479,yes,no,unknown,28,may,2,-1,0,unknown


In [8]:
x_train = pd.get_dummies(x_train, drop_first = True)
x_test = pd.get_dummies(x_test, drop_first = True)

In [9]:
x_train.describe()

,age,balance,day,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,job_management,...,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown
count,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,...,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000
mean,41.091142,1483.774437,15.641103,2.659130,45.555478,0.688967,0.199767,0.029759,0.025952,0.222067,...,0.141647,0.115307,0.018415,0.279254,0.085004,0.026263,0.021290,0.044678,0.065657,0.782440
std,11.305560,3311.055181,8.368983,2.863507,104.449411,2.049696,0.399841,0.169929,0.158998,0.415652,...,0.348702,0.319404,0.134452,0.448650,0.278898,0.159922,0.144354,0.206603,0.247690,0.412603
min,18.000000,-6847.000000,1.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.000000,102.000000,8.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,39.000000,515.000000,16.000000,2.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,49.000000,1591.750000,21.000000,3.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,95.000000,102127.000000,31.000000,43.000000,854.000000,58.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
columns = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']
x_train[columns] = scaler.fit_transform(x_train[columns])
x_test[columns] = scaler.transform(x_test[columns])


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x_train,y_train,random_state = 42, test_size = 0.2)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train,Y_train)

pred_tr = rf.predict_proba(X_train)[:,1]
pred_te = rf.predict_proba(X_test)[:,1]

print(roc_auc_score(Y_train,pred_tr))
print(roc_auc_score(Y_test,pred_te))

0.9999999999999999
0.7761590028927434


In [13]:
pred_test = rf.predict_proba(x_test)[:,1]
pd.DataFrame({'ID':test_id, 'price': pred_test}).to_csv('0000.csv',index = False)
result = pd.read_csv('0000.csv')

In [14]:
result

,ID,price
0,53608,0.79
1,51055,0.90
2,52573,0.09
3,50458,0.15
4,52272,0.30
...,...,...
4286,50332,0.22
4287,51293,0.84
4288,54023,0.16
4289,53775,0.53


In [15]:
y_test = pd.read_csv('../input/bank-marketing-response-predict/test_label.csv')
y_test

,ID,y
0,53608,yes
1,51055,yes
2,52573,no
3,50458,no
4,52272,no
...,...,...
4286,50332,no
4287,51293,yes
4288,54023,no
4289,53775,yes


In [16]:
y_test['y'] = y_test.y.map(lambda x : 1 if x=='yes' else 0)


In [17]:
print(roc_auc_score(y_test.y,result.price))

0.7809573102594688
